<center>

<font size="5">MGMTMSA 408: Operations Analytics</font>

</center>


<center>

<font size="5">Assignment 1: LP Duality and Revenue Management</font>

</center>


<div style="display: flex; justify-content: space-between;">
    <div style="text-align: left;">
        Due on BruinLearn by 1:00pm on April 15, 2024.
    </div>
    <div style="text-align: right;">
        Arnav Garg (906310841) <br>
    </div>
</div>


**<font size="5">1. Semiconductor Manufacturing</font>**

A specialized semiconductor manufacturing plant produces three types of semiconductor chips. Each chip requires a certain number of germanium (Ge) transistors and a certain number of silicon (Si) transistors. In addition, each chip requires a certain amount of manufacturing time. Each chip can be sold for a certain unit price. The requirements and prices of the chips are provided below:

Chip Type | No. of Ge Transistors | No. of Si Transistors | Manufacturing Time | Price
--- | --- | --- | --- | ---
1 | 14 | 30 | 20 | 500
2 | 20 | 20 | 30 | 800
3 | 40 | 15 | 50 | 1000

In the above, the required number of transistors is presented in units of billions (e.g., the value of
14 for chip type 1 means that 14 billion germanium transistors are required). The manufacturing
time requirement is given in minutes, and the price is given in dollars.
The manufacturing plant has a total of 300 billion Ge transistors and 200 billion Si transistors
available. The plant also has 18 hours of manufacturing time available.

**<font size="4">Part 1: LP Formulation</font>**

The plant is interested in deciding how much to produce of each type of chip, so as to maximize its
revenue.

a) Formulate the production problem as a linear program. What are the decision variables? What
is the objective function? What are the constraints?

\begin{align}
\text{maximize} \;\; & 500x_1 + 800x_2 + 1000x_3 \;\;\;\; \text{[Objective Function]}\\
\text{s.t.} \;\; & 14x_1 + 20x_2 + 40x_3 \le 300 \;\;\;\; \text{[Germanium Constraint]}\\
            & 30x_1 + 20x_2 + 15x_3 \le 200 \;\;\;\; \text{[Silicon Constraint]}\\
            & 20x_1 + 30x_2 + 50x_3 \le 1080 \;\;\;\; \text{[Time Constraint]}\\
            & x_i \ge 0 \quad i = 1 \ldots 3 \;\;\;\; \text{[Non-Negativity Constraint]}\\
\end{align}


where decision variables are given as follows:

$x_1$ is the number of chips produced of type 1

$x_2$ is the number of chips produced of type 1

$x_3$ is the number of chips produced of type 1

to maximize revenue

b) Implement your linear program in Python using Gurobi.
What is the optimal objective value?

In [17]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *
import numpy as np
import pandas as pd

##############################################################################################
# Initialize the model 
primal = Model()

# Set the OutputFlag parameter to 0 to disable logging
primal.Params.outputflag = 0

##############################################################################################
# Define Variables
x1 = primal.addVar(lb = 0)
x2 = primal.addVar(lb = 0)
x3 = primal.addVar(lb = 0)

##############################################################################################
# Define Constraints
## Germanium Transistors
Ge_constr = primal.addConstr(14 * x1 + 20 * x2 + 40 * x3 <= 300)

## Silicon Transistors
Si_constr = primal.addConstr(30 * x1 + 20 * x2 + 15 * x3 <= 200)

## Time Constraint
t_constr = primal.addConstr(20 * x1 + 30 * x2 + 50 * x3 <= 1080)

##############################################################################################
# Construct Objective
primal.setObjective(500 * x1 + 800 * x2 + 1000 * x3, GRB.MAXIMIZE)

##############################################################################################
# Update and solve
primal.update()
primal.optimize()

##############################################################################################
# Print Optimal Solution
print("Maximum Profit(Optimal Objective Value): $", primal.objVal)

##############################################################################################

Maximum Profit(Optimal Objective Value): $ 9600.0


c) What are the optimal values of the decision variables?

In [18]:
print("Optimal Values of Decision Variables:")
print("Chip Type 1:", x1.x)
print("Chip Type 2:", x2.x)
print("Chip Type 3:", x3.x)

Optimal Values of Decision Variables:
Chip Type 1: 0.0
Chip Type 2: 7.0
Chip Type 3: 4.0


**<font size="4">Part 2: Dual Formulation</font>**

a) Write down the dual of the problem in Part 1. What are the decision variables? What is the
objective function? What are the constraints?

\begin{align}
\text{minimize} \;\; & 300w_{Ge} + 200w_{Si} + 1080w_t \;\;\;\; \text{[Objective Function]} \\
\text{s.t.} \;\; & 14w_{Ge} + 30w_{Si} + 20w_t \ge 500 \;\;\;\; \text{[Weight of ChipType1 Constraint]} \\
            & 20w_{Ge} + 20w_{Si} + 30w_t \ge 800 \;\;\;\; \text{[Weight of ChipType2 Constraint]} \\
            & 40w_{Ge} + 15w_{Si} + 50w_t \ge 1000 \;\;\;\; \text{[Weight of ChipType3 Constraint]} \\
            & w_{Ge}, w_{Si}, w_t \ge 0 \;\;\;\; \text{[Non-Negativity Constraint]}\\
\end{align}


where decision variables are given as follows:

$w_{Ge}$ is the weight of the Germanium constraint

$w_{Si}$ is the weight of the Germanium constraint

$w_{t}$ is the weight of the Germanium constraint

to get the lowest bound on revenue

b) Implement your dual linear program in Python using Gurobi and solve it.
What is the optimal objective value?

In [19]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *
import numpy as np
import pandas as pd

##############################################################################################
# Initialize the model 
dual = Model()

# Set the OutputFlag parameter to 0 to disable logging
dual.Params.outputflag = 0

##############################################################################################
# Define Variables
w_Ge = dual.addVar(lb = 0)
w_Si = dual.addVar(lb = 0)
w_t = dual.addVar(lb = 0)

##############################################################################################
# Define Constraints
## Weight of chip type 1 constraint
chip1_constr = dual.addConstr(14 * w_Ge + 30 * w_Si + 20 * w_t >= 500)

## Weight of chip type 2 constraint
chip2_constr = dual.addConstr(20 * w_Ge + 20 * w_Si + 30 * w_t >= 800)

## Weight of chip type 3 constraint
chip3_constr = dual.addConstr(40 * w_Ge + 15 * w_Si + 50 * w_t >= 1000)

##############################################################################################
# Construct Objective
dual.setObjective(300 * w_Ge + 200 * w_Si + 1080 * w_t, GRB.MINIMIZE)

##############################################################################################
# Update and solve
dual.update()
dual.optimize()

##############################################################################################
# Print Optimal Solution
print("Lowest Bound on Profit(Optimal Objective Value): $", dual.objVal)

##############################################################################################

Lowest Bound on Profit(Optimal Objective Value): $ 9600.0


c) What are the optimal values of the dual variables?

In [20]:
print("Optimal Values of Dual Variables:")
print("Weight of Chip Type 1:", w_Ge.x)
print("Weight of Chip Type 2:", w_Si.x)
print("Weight of Chip Type 3:", w_t.x)

Optimal Values of Dual Variables:
Weight of Chip Type 1: 16.0
Weight of Chip Type 2: 24.0
Weight of Chip Type 3: 0.0


d) Verify that the optimal values you obtain here are the same as the shadow prices of the constraints
in the formulation you implemented in Part 1, which you can access by using the .pi attribute
of a Gurobi constraint reference.

In [21]:
print("Shadow Prices:")
print("Germanium Transistors:", Ge_constr.pi)
print("Silicon Transistors:", Si_constr.pi)
print("Time:", t_constr.pi)

Shadow Prices:
Germanium Transistors: 16.0
Silicon Transistors: 24.0
Time: 0.0


I verify that the optimal values I obtain in Part2c are the same as the shadow prices of the constraints
in the formulation I implemented in Part1a.

**<font size="4">Part 3: Shadow Prices / Marginal Analysis</font>**

a) To increase the plant’s revenue, a plant manager suggests increasing the amount of manufacturing
time by 10 hours. Will this change result in an improvement in revenue? Explain your answer.

Increasing the amount of manufacturing time by 10 hours will not result in an improvement in revenue because the shadow price for time is 0. This means the time constraint is redundant in this scenario.

b) A different plant manager proposes increasing the amount of available Ge transistors by 10
billion. Based on the shadow prices/optimal dual variables, what is your prediction of how
much the revenue will change from your answer in Part 1-b)?

Based on the shadow prices, if the amount of available Ge transistors by 10 billion, the revenue will increase by \$160 to \$9760 as compared to the answer in part 1b.

c) Verify your answer to part b) by solving the primal LP from Part 1 with the right-hand side of
the Ge transistor constraint increased by 10.
(You can do this by reformulating the problem from scratch with a different right-hand side
value for the Ge transistor constraint. Another way is to use the following lines of code:

Ge_constr.rhs = 310

m.update()

m.optimize()

where Ge_constr is the reference to your Ge constraint, and m is the Gurobi model object. The
.rhs attribute of a constraint accesses the right-hand side / constant part of the constraint,
allowing you to modify your problem after you have built it.)

In [22]:
Ge_constr.rhs = 310
primal.update()
primal.optimize()
print("Maximum Profit: $", primal.objVal)

Maximum Profit: $ 9760.0


We obtain the same answer as expected.

d) Suppose now that instead of increasing the amount of available Ge transistors by 10 billion, we
consider increasing it by 300 billion. Based on the shadow prices, what is your prediction of how
much the objective will change from your answer in Part 1-b)?

Based on the shadow prices, if the amount of available Ge transistors by 300 billion, the revenue will increase by \$4800 to \$14200 as compared to the answer in part 1b.

e) Check your answer in d) by solving the primal LP from Part 1 with the right-hand side of the
Ge constraint increased by 300. What is the change in revenue from your answer in Part 1-b)?
(You will find that it is not the same as the value in d).) What explains this discrepancy?

In [23]:
Ge_constr.rhs = 600
primal.update()
primal.optimize()
print("Maximum Profit: $", primal.objVal)

Maximum Profit: $ 13333.333333333334


This discrepancy can be explained by the fact that shadow prices are valid for small changes in the constraint's value. However, after some threshold, that constraint might no longer be binding, hence it will not have the same marginal impact when increased.

f) Lastly, suppose that instead of increasing only the amount of available Ge transistors, we consider
increasing the amount of available Ge transistors by 10 billion and the amount of Si transistors
by 20 billion. What is your prediction of the change in the objective from your answer in Part
2-b)?

Based on the shadow prices, if the amount of available Ge transistors by 10 billion and Si transistors by 20 billion, the revenue will increase by \$640 to \$10240 as compared to the answer in part 1b.

g) Check your answer in f) by solving the primal LP from Part 1 with the right-hand side of the Ge
constraint increased by 10 and the right-hand side of the Si constraint increased by 20. What is
the change in revenue from your answer in Part 1-b)?

In [24]:
Ge_constr.rhs = 310
Si_constr.rhs = 220
primal.update()
primal.optimize()
print("Maximum Profit: $", primal.objVal)

Maximum Profit: $ 10240.000000000002


We obtain the same answer as expected.

**<font size="5">2. Cloud Computing</font>**

Cirrus is a cloud computing provider that offers its users the ability to reserve computational
capacity on its cloud. In this problem, we will analyze how Cirrus should allocate its capacity.
Cirrus sells its computational resources in units called instances. These are virtual computers
that a user can reserve for a whole day. Each instance consist of three components:

• Central processing units (CPUs): each instance reserves some number of CPUs, range from 1
CPU to 64 CPUs. \
• Memory: each instance reserves some amount of memory, ranging from 1 GB to 128GB, in
increments of 1 GB. \
• Graphical processing units (GPUs): each instance reserves some amount of GPU memory,
ranging from 1 GB to 8 GB.

In total, on a given day, Cirrus has 512 CPUs, 1024 GB of memory and 64 GB of GPU memory
available.
Based on its user requirements, offers the following instances:

Instance | Name | CPU(#) | Memory(GB) | GPU(GB) | Price | Rate (#/day)
--- | --- | --- | --- | --- | --- | ---
1 | C1 | 16 | 8 | 1 | $7 | 5.0
2 | C2 | 32 | 16 | 1 | $12 | 5.0
3 | C3 | 64 | 32 | 1 | $24 | 1.8
4 | M1 | 8 | 32 | 1 | $22 | 3.0
5 | M2 | 16 | 64 | 1 | $44 | 2.6
6 | M3 | 32 | 128 | 1 | $88 | 1.0
7 | G1 | 16 | 16 | 2 | $30 | 0.8
8 | G2 | 32 | 32 | 6 | $90 | 0.4
9 | G3 | 64 | 64 | 8 | $120 | 0.3

Columns 3 - 5 specify the hardware requirements of each instance type. Column 6 specifies the
price for activating the instance. We will also assume that an instance is reserved for a whole day.

Column 7 specifies the arrival rate. This is the number of users requesting an instance of this
type each day. In this problem, we’ll assume that the demand for each type of instance follows
a Poisson arrival process, with the arrival rate as given in the table above. We will assume that
Cirrus only allows reservations for a period of 5 days. We will also assume that this period of 5 days
precedes the day for which the instance is reserved and is exclusive of this day (i.e., the requests
are not fulfilled during the reservation period). For example, instance requests can come in on
Monday, Tuesday, Wednesday, Thursday and Friday, at any time on those days; the instances that
are accepted are then reserved for the whole day of Saturday. Instances are not reserved during the
5 day reservation period.

Cirrus is interested in understanding how it should allocate its limited capacity to the different
types of instances. Cirrus currently accepts requests in a first-come first-serve fashion. In this
problem, we will approach Cirrus’s problem from a revenue management lens.

**<font size="4">Part 1: Capacity control formulationn</font>**

In this first part of the problem, we need to mathematically formulate the capacity control problem
for Cirrus as a linear optimization problem. Let x1, . . . , x9 be the number of instances that are
reserved of each instance type. These will be the decision variables of our problem.

a) Based on the prices given in the table earlier, what is the objective function of the problem?

\begin{align}
\text{maximize} \;\; & 7x_1 + 12x_2 + 24x_3 + 22x_4 + 44x_5 + 88x_6 + 30x_7 + 90x_8 + 120x_9 \\
\end{align}


b) In terms of x1, . . . , x9, what is the constraint on the total memory usage of the instances that
are reserved?

\begin{align}
\text{s.t.} \;\; & 8x_1 + 16x_2 + 32x_3 + 32x_4 + 64x_5 + 128x_6 + 16x_7 + 32x_8 + 64x_9 \le 1024\\
\end{align}


c) Let’s now consider the forecasted demand for each instance type. Recall that for a Poisson
arrival process with rate λ per unit time, the total number of arrivals in a period of length T is
a Poisson random variable Y with mean λT .
Over the five day period, what is the expected number of requests of instance type 5? Assuming
there will be exactly this many requests over the T = 5 day period, what is the constraint on
the number of requests of instance type 5 we may accept?

Over the five day period, the expected number of requests of instance type 5 is 13. The constraint on the number of requests of instance type 5 we may accept is given as follows:

$\text{Expected Number of Requests} = \lambda_5T = \text{Rate} \times \text{Time} = 2.6 \times 5 = 13$

$\text{s.t.} \;\; x_5 \le 13 \;\;\;\; \text{[Constraint]}$

d) Using the logic in (a) – (c), write down the mathematical formulation of the T = 5 day capacity
control problem as a linear optimization problem.

\begin{align}
\text{maximize} \;\; & 7x_1 + 12x_2 + 24x_3 + 22x_4 + 44x_5 + 88x_6 + 30x_7 + 90x_8 + 120x_9 \\
\text{s.t.} \;\; & 16x_1 + 32x_2 + 64x_3 + 8x_4 + 16x_5 + 32x_6 + 16x_7 + 32x_8 + 64x_9 \le 512 \\
& 8x_1 + 16x_2 + 32x_3 + 32x_4 + 64x_5 + 128x_6 + 16x_7 + 32x_8 + 64x_9 \le 1024 \\
& 1x_1 + 1x_2 + 1x_3 + 1x_4 + 1x_5 + 1x_6 + 2x_7 + 6x_8 + 8x_9 \le 64 \\
& x_1 \ge 0, x_1 \le 25 \\
& x_2 \ge 0, x_2 \le 25 \\
& x_3 \ge 0, x_3 \le 9 \\
& x_4 \ge 0, x_4 \le 15 \\
& x_5 \ge 0, x_5 \le 13 \\
& x_6 \ge 0, x_6 \le 5 \\
& x_7 \ge 0, x_7 \le 4 \\
& x_8 \ge 0, x_8 \le 2 \\
& x_9 \ge 0, x_9 \le 1.5
\end{align}


**<font size="4">Part 2: Solving the capacity control problem in Python/Gurobi</font>**

Now, implement your formulation in Part 1(d) using Python and Gurobi.

a) Solve the problem. What is the optimal objective value?

In [25]:
##############################################################################################
#Importing necessary libraries
from gurobipy import *
import numpy as np
import pandas as pd

##############################################################################################
# Initialize the model 
m = Model()

# Set the OutputFlag parameter to 0 to disable logging
m.Params.outputflag = 0

##############################################################################################
# Define Variables
x = m.addVars(9, lb = 0)

##############################################################################################
# Define Constraints
## CPU Constraint
cpu = np.array([16, 32, 64, 8, 16, 32, 16, 32, 64])
CPU_constr = m.addConstr(sum(cpu[i]*x[i] for i in range(len(cpu))) <= 512)

## Memory Constraint
memory = np.array([8, 16, 32, 32, 64, 128, 16, 32, 64])
Mem_constr = m.addConstr(sum(memory[i]*x[i] for i in range(len(memory))) <= 1024)

## GPU Constraint
gpu = np.array([1, 1, 1, 1, 1, 1, 2, 6, 8])
GPU_constr = m.addConstr(sum(gpu[i]*x[i] for i in range(len(gpu))) <= 64)

## Rate Constraint
rate = np.array([25, 25, 9, 15, 13, 5, 4, 2, 1.5])
for i in range(len(rate)):
    m.addConstr(x[i] <= rate[i])

##############################################################################################
# Construct Objective
revenue = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])
m.setObjective(sum(revenue[i]*x[i] for i in range(len(revenue))), GRB.MAXIMIZE)

##############################################################################################
# Update and solve
m.update()
m.optimize()

##############################################################################################
# Print Optimal Solution
print("Maximum Revenue(Optimal Objective Value): $", m.objVal)

##############################################################################################

Maximum Revenue(Optimal Objective Value): $ 1039.4285714285716


b) In the optimal allocation, how many requests of instance type C1 are accepted?

In [26]:
print("Optimal Allocation:")
for i in range(len(revenue)):
    print("Instance Type", i+1, ":", x[i].x)

Optimal Allocation:
Instance Type 1 : 6.285714285714286
Instance Type 2 : 0.0
Instance Type 3 : 0.0
Instance Type 4 : 3.428571428571429
Instance Type 5 : 0.0
Instance Type 6 : 5.0
Instance Type 7 : 4.0
Instance Type 8 : 2.0
Instance Type 9 : 1.5


We observe that in the optimal allocation, ~6.29 requests of instance type C1 are accepted.

**<font size="4">Part 3: Simulating current practice</font>**

Cirrus would like to understand how well its current policy does. Currently, Cirrus simply accepts
requests in a first-come first-serve fashion, without considering the revenue of the requests. In this
part of the problem, we will simulate Cirrus’s current policy.

Set your random seed to 10 using the random.seed() function in numpy. Using the provided
function generateArrivalSequences in the HW1 - Cloud Code.ipynb notebook, generate 100 se-
quences of request arrivals, using the rates provided in the table above, over a period of T = 5 days.
This function will generate an array of arrays:

• arrival sequence times: This contains the time at which each request arrives, in the interval
[0, 5]. For example, arrival sequence times[0][1] is the time at which the second request
arrives in the first sequence. \
• arrival sequence types: The instance type of each request. (Note: the instance types in
this array are numbered from 0 to 8, in accordance with how Python’s numbering starts at
zero. Thus, for example, a value of 5 in this array indicates instance type 6 / M3.)

If you have done this step correctly, the first three times in arrival sequences times[0] should
be 0.07414243 0.1246028 0.15928449.

In [27]:
# Preconditions:
# nSimulations = integer specifying number of simulations to run
# rates = array containing arrival rate (# / day) for each of the instance
# types (should be an array with 9 elements)
# T = length of horizon in days.

def generateArrivalSequences( nSimulations, rates, T ):
    total_rate = sum(rates)
    nTypes = len(rates)

    arrival_sequences_times = []
    arrival_sequences_types = [];

    for s in range(nSimulations):
        single_arrival_sequence_time = [];
        single_arrival_sequence_type = [];
        t = 0;
        while (t < T):
            single_time = np.random.exponential(1.0/total_rate)
            single_type = np.random.choice(nTypes, p = rates/total_rate)

            t += single_time;

            if (t < T):
                single_arrival_sequence_time.append(t)
                single_arrival_sequence_type.append(single_type)
            else:
                break

        arrival_sequences_times.append(np.array(single_arrival_sequence_time))
        arrival_sequences_types.append(np.array(single_arrival_sequence_type))
    return arrival_sequences_times, arrival_sequences_types



# Code to test out above function
np.random.seed(1)
nSimulations_test = 100
rates_test = np.array([5.0, 2.0, 3.0])
T_test = 8
times, types = generateArrivalSequences(nSimulations_test, rates_test, T_test)

# If code above is working correctly, code below should show
# value of 80.71:
counts = np.array([len(types[i]) for i in range(nSimulations_test)] )
counts.mean()

80.71

In [28]:
np.random.seed(10)
nSimulations = 100
rates = np.array([5, 5, 1.8, 3, 2.6, 1, 0.8, 0.4, 0.3])
T = 5
times, types = generateArrivalSequences(nSimulations, rates, T)
times[0]

array([0.07414243, 0.1246028 , 0.15928449, 0.1703762 , 0.17968569,
       0.23779251, 0.23799131, 0.32214315, 0.38642729, 0.51196926,
       0.55126954, 0.57475458, 0.60405595, 0.65238384, 0.70519578,
       0.78740185, 0.90765536, 0.91241996, 0.9185014 , 0.92091505,
       0.96077223, 0.97191941, 0.99369472, 1.01132902, 1.03111801,
       1.05616581, 1.14264621, 1.16700269, 1.38771201, 1.47562187,
       1.52133751, 1.55976805, 1.56178183, 1.56595073, 1.58612921,
       1.58817843, 1.60718495, 1.62855072, 1.73506112, 1.84081623,
       1.88756662, 1.93334033, 1.95132628, 1.96946998, 2.01044946,
       2.04284187, 2.04617896, 2.06703689, 2.25677443, 2.27602698,
       2.3193975 , 2.32294156, 2.39947557, 2.44379428, 2.46549244,
       2.48339641, 2.53963883, 2.54224165, 2.54798842, 2.65703572,
       2.66944199, 2.67512209, 2.70700517, 2.71503145, 2.76708375,
       2.78144725, 2.79437102, 2.92271451, 2.96474668, 2.96549902,
       3.0082648 , 3.04969828, 3.09379581, 3.10187596, 3.16136

a) What is the average number of arrivals of type C1 in the set of simulated sequences?

In [29]:
f = 0
for i in range(nSimulations):
    f += sum(types[i] == 0) #c1 is 0
print("Average number of arrivals of type C1 =", f/nSimulations)

Average number of arrivals of type C1 = 26.63


b) What is the average number of arrivals, of all types, over the set of simulated sequences? Does
this make sense? (Hint: What is the expected value of the sum of Poisson random variables?)

In [30]:
instance_types = ['C1', 'C2', 'C3', 'M1', 'M2', 'M3', 'G1', 'G2', 'G3']
avg_arrivals = np.zeros(len(instance_types))

for j in range(len(instance_types)):
    for i in range(nSimulations):
        avg_arrivals[j] += sum(types[i] == j)
    print("Average number of arrivals of type", instance_types[j], "=", avg_arrivals[j]/nSimulations)
print("Average number of arrivals of all types =", sum(avg_arrivals)/nSimulations)

Average number of arrivals of type C1 = 26.63
Average number of arrivals of type C2 = 24.38
Average number of arrivals of type C3 = 9.01
Average number of arrivals of type M1 = 14.88
Average number of arrivals of type M2 = 13.32
Average number of arrivals of type M3 = 4.93
Average number of arrivals of type G1 = 4.2
Average number of arrivals of type G2 = 2.27
Average number of arrivals of type G3 = 1.38
Average number of arrivals of all types = 101.0


The average number of arrivals, of all types, over the set of simulated sequences is 101.
The expected value of the sum of Poisson random variables is the sum of the expected values of the individual random variables. The expected value of a Poisson random variable is its rate parameter, so the expected value of the sum of Poisson random variables is the sum of individual rate parameters which comes out to be 19.9 in this case. Hence, over a period of 5 days, the expected value for average number of arrivals of all types is 99.5 which is approximately what we get.

$\lambda_{all} = \sum_{i=0}^{9}\lambda_i = 19.9$ \
$\text{Expected Value} = \lambda_{all} * T = 19.9*5 = 99.5$

c) Next, implement Cirrus’s current policy. This policy accepts any request, so long as there is
capacity for it. You may use the code skeleton given in HW1 - Cloud Code.ipynb as a starting
point.
What is the average revenue garnered by this policy over the 100 simulated sequences?

In [31]:
# Preconditions for code below:
nSimulations = 100
nResources = 3
B = np.array([512, 1024, 64])
arrival_sequences_times = times
arrival_sequences_types = types

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy()
    single_revenue = 0.0 # will contain the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s])

    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (i)
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is sufficient capacity for the request
        if (b[0] >= cpu[i]) and (b[1] >= memory[i]) and (b[2] >= gpu[i]):
            # If there is sufficient capacity, accrue the revenue
            single_revenue += revenue[i]
            # and remove the cpu capacity
            b[0] -= cpu[i]
            # and remove the memory capacity
            b[1] -= memory[i]
            # and remove the gpu capacity
            b[2] -= gpu[i]

    # Save the results of this simulation here
    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[:, s] = b

# Find the average revenue
revenue_myopic = results_myopic_revenue.mean()

# Find the average remaining quantity of each resource
remaining_cpu_myopic = results_myopic_remaining_capacity[0, :].mean()
remaining_memory_myopic = results_myopic_remaining_capacity[1, :].mean()
remaining_gpu_myopic = results_myopic_remaining_capacity[2, :].mean()

# Print results
print("Average Revenue: $", revenue_myopic)

Average Revenue: $ 528.28


d) What is the average remaining capacity (of CPUs, memory and GPUs) of this policy?

In [32]:
# Print results
print("Average Remaining CPU Capacity: ", remaining_cpu_myopic)
print("Average Remaining Memory Capacity: ", remaining_memory_myopic)
print("Average Remaining GPU Capacity: ", remaining_gpu_myopic)

Average Remaining CPU Capacity:  0.24
Average Remaining Memory Capacity:  371.52
Average Remaining GPU Capacity:  37.42


**<font size="4">Part 4: A bid-price control policy</font>**

Let’s now develop a bid-price control policy, using the solution of the LP.

a) As a warm-up, suppose that we receive a request for instance type 5 (M2). Let π1, π2, π3 be the
shadow prices / dual values of the constraints for CPUs, memory and GPUs at a particular point
in time. In terms of the shadow prices, what is the approximate opportunity cost of accepting
this request?

Opportunity Cost = $16\pi_1 + 64\pi_2 + 1\pi_3$

b) As a further warm-up, suppose that we receive a request at time t. What is the expected number
of requests of type i we will receive from time t to time T (including both t and T )?

Expected # of Requests = $\lambda_i(T-t+1)$

c) Next, implement Cirrus’s current policy. This policy accepts any request, so long as there is
capacity for it. You may use the code skeleton given in HW1 - Cloud Code.ipynb as a starting
point.
What is the average revenue garnered by this policy over the 100 simulated sequences?

In [33]:
# Preconditions for code below:
nSimulations = 100
nInstances = 9
nResources = 3
B = np.array([512, 1024, 64])
arrival_sequences_times = times
arrival_sequences_types = types

# As we did in-class, define a function bpc() to re-solve the LO each time:
def bpc(b, t):
    # for r in range(nResources):
    #     # Set the RHS of the resource constraint to b[r] here
    CPU_constr.rhs = b[0]
    Mem_constr.rhs = b[1]
    GPU_constr.rhs = b[2]

    for k in range(nInstances):
        # Set the RHS of the forecast constraint for each instance
        # type to the expected number of requests over the duration
        # of the remaining horizon (T - t).
        # ...
        x[k].ub = rates[k] * (5 - t)

    # Re-solve the model:
    m.update()
    m.optimize()

    # Obtain the dual values/shadow prices
    dual_val = cpu[g]*CPU_constr.pi + memory[g]*Mem_constr.pi + gpu[g]*GPU_constr.pi

    # Return the dual values:
    return dual_val


results_bpc_revenue = np.zeros(nSimulations)
results_bpc_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy()
    single_revenue = 0.0 # will contain the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s])

    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (g)
        arrival_time = arrival_sequences_times[s][j]
        g = arrival_sequences_types[s][j]

        # Check if there is enough capacity
        if (b[0] >= cpu[g]) and (b[1] >= memory[g]) and (b[2] >= gpu[g]):
            # Re-solve the LO and obtain the dual values
            dual_val = bpc(b, arrival_time)

            # Check if the revenue is at least the sum of the bid prices:
            if (revenue[g] >= dual_val):
                # If there is sufficient capacity, accrue the revenue
                single_revenue += revenue[g]
                # and remove the cpu capacity
                b[0] -= cpu[g]
                # and remove the memory capacity
                b[1] -= memory[g]
                # and remove the gpu capacity
                b[2] -= gpu[g]

    # Save the results of this simulation here:
    results_bpc_revenue[s] = single_revenue
    results_bpc_remaining_capacity[:, s] = b


# Find the average revenue
revenue_bpc = results_bpc_revenue.mean()

# Find the average remaining quantity of each resource
remaining_cpu_bpc = results_bpc_remaining_capacity[0, :].mean()
remaining_memory_bpc = results_bpc_remaining_capacity[1, :].mean()
remaining_gpu_bpc = results_bpc_remaining_capacity[2, :].mean()

# Print results
print("Average Revenue: $", revenue_bpc)

Average Revenue: $ 925.59


d) What is the average remaining capacity (of CPUs, memory and GPUs) of this policy?

In [34]:
# Print results
print("Average Remaining CPU Capacity: ", remaining_cpu_bpc)
print("Average Remaining Memory Capacity: ", remaining_memory_bpc)
print("Average Remaining GPU Capacity: ", remaining_gpu_bpc)

Average Remaining CPU Capacity:  27.2
Average Remaining Memory Capacity:  4.88
Average Remaining GPU Capacity:  20.62
